In [ ]:
pip install --upgrade google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.59.0
    Uninstalling google-cloud-aiplatform-1.59.0:
      Successfully uninstalled google-cloud-aiplatform-1.59.0


In [ ]:
from google.cloud import aiplatform

In [ ]:

import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [ ]:

PROJECT_ID = "prime-hydra-428901-s6"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from vertexai.generative_models import GenerativeModel
model = GenerativeModel("gemini-1.5-pro-001")

In [ ]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information, including live exchange rates. 

To get the most up-to-date exchange rate for euros to dollars, I recommend checking a reliable online source such as:

* **Google Finance:** Search "EUR to USD" on Google 
* **XE.com:** This website specializes in currency conversions.
* **Your Bank's Website:** Most banks provide current exchange rates for their customers.

Remember that exchange rates fluctuate constantly, so the rate you see at any given moment may change. 



In [ ]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content("""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt))
print(response.text)

```json
{
  "from": "euros",
  "to": "US dollars"
}
``` 



In [ ]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

# model = GenerativeModel("gemini-1.5-pro-001")

In [ ]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [ ]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

In [ ]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [ ]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
    }
  }
}



In [ ]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'date': 'latest', 'to': 'SEK', 'from': 'AUD'}

In [ ]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"AUD","date":"2024-07-24","rates":{"SEK":7.1144}}'

In [ ]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

"Let's break down the conversion step-by-step.\n\n**1. Fetch the Exchange Rate**\n\nWe need the latest exchange rate for Australian Dollars (AUD) to Swedish Krona (SEK). I got the exchange rate for you:  1 AUD is equal to 7.1197 SEK. This is based on the exchange rate from July 23rd, 2024.\n\n**2. Calculation**\n\nNow we'll convert 500 AUD to SEK using the exchange rate:\n\n*   500 AUD * 7.1197 SEK/AUD = 3559.85 SEK\n\n**3. Result**\n\nTherefore, as of July 23rd, 2024, 500 Australian Dollars is equivalent to **3559.85 Swedish Krona**. \n"

Parallel Function Calling

In [ ]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona and GBP to Swedish Krona?
How much is 500 Australian dollars worth in Swedish krona? How much is 700 GBP worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [ ]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
    }
  }
}
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "GBP"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
    }
  }
}

